- for exporting erddap data to geojson for plotting via leaflet
- used for NRT maps on website

In [36]:
import pandas as pd
import datetime 

from erddapy import ERDDAP
import pandas as pd
import numpy as np

In [38]:
server_url = 'http://downdraft.pmel.noaa.gov:8080/erddap'


d = ERDDAP(server=server_url,
           protocol='tabledap',
           response='csv',
          )

d.dataset_id='sg401_EcoFOCI_2019'

d.variables =  ['longitude','latitude','ctd_depth','profileid']

d.constraints = {
'ctd_depth<=':1
}

d.get_download_url()

'http://downdraft.pmel.noaa.gov:8080/erddap/tabledap/sg401_EcoFOCI_2019.csv?longitude,latitude,ctd_depth,profileid&ctd_depth<=1'

In [39]:
df = d.to_pandas(
    skiprows=(1,)  # units information can be dropped.
).dropna()

df.sort_index(inplace=True)
df.columns = [x[1].split()[0] for x in enumerate(df.columns)]

df.tail()

,longitude,latitude,ctd_depth,profileid
16262,-174.029643,61.734239,0.057159,1020
16263,-174.029642,61.734239,-0.056761,1020
16264,-174.029642,61.734239,-0.170680,1020
16265,-174.029642,61.734239,-0.255012,1020
16266,-174.029642,61.734239,-0.255012,1020


In [40]:
print(df.groupby('profileid').first())

            longitude   latitude  ctd_depth
profileid                                  
1         -173.598569  60.426905   0.326673
2         -173.608655  60.426610   0.942308
3         -173.612110  60.427173   0.756084
4         -173.615397  60.429410   0.912789
5         -173.619142  60.434414   0.991975
...               ...        ...        ...
1015      -174.057032  61.740910   0.972427
1016      -174.057032  61.740910   0.965673
1017      -174.057032  61.740910  -0.275147
1018      -174.032585  61.734652   0.990983
1020      -174.032585  61.734652   0.092776

[607 rows x 3 columns]


In [41]:
import geojson

def data2geojson(df):
    df['profileid']=df.index
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["longitude"],
                                                    X["latitude"],
                                                    X["ctd_depth"])),
                            properties=dict(name=X['profileid'])))
    df.apply(insert_features, axis=1)
    with open('map1.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

data2geojson(df.groupby('profileid').first())